In [208]:
# pip install -q -U google-generativeai


Agent test

In [209]:
import os
parent_dir = os.path.dirname(os.getcwd())
file_path = os.path.join(parent_dir, 'DB', 'testDatabase.txt')

data = ""
# Read the file
with open(file_path, 'r') as f:
    data = f.read()

print(data)

#1 Long black
#2 Machiatto
#3 Iced Caramel Latte
#4 Flat White
#5 Latte
#6 Cappucino
#7 3/4 Double Ristretto Flat White


In [210]:
# prompt generation
def generatePrompt(DATA: str, USER_ORDER: str) -> str:
    return f"""
    You are a coffee order assistant that maps customer orders to standard menu items.
    Given a customer order, check the menu and provide the exact corresponding output.

    Menu:
    {DATA}

    Input Order: {USER_ORDER}
    Output Format: [menu_number] [size] [menu_item] [short additional notes: e.g. 3/4, how many sugars]

    Rules:
    - guess the coffee order output
    - melbourne magic is a double ristretto flat white
    """

In [211]:
def generateJudgePrompt(output: str, DATA: str, USER_ORDER: str) -> str:
    return f"""You are a precise coffee order validator that evaluates the accuracy of coffee order mappings. Your role is to judge whether the mapped order matches the customer's intent.

Menu:
{DATA}

Original Customer Order: {USER_ORDER}
Coffee agent guess: {output}

Instructions:
1. Analyze if the mapped order accurately represents the customer's intent
2. Consider common coffee order variations and aliases
3. Account for regional naming conventions (e.g., Melbourne Magic = Double Ristretto Flat White)
4. Check for specific customizations (size, strength, milk type)

Evaluation Criteria:
- Base drink match (e.g., latte, cappuccino, flat white)
- Drink size if specified
- Coffee strength (single/double shot, ristretto)
- Milk type if specified
- Temperature preference if mentioned
- Additional modifications (extra hot, no foam, etc.)

""" + """Output Format:
{
    "is_valid": boolean,
    "confidence": float (0.0-1.0),
    "explanation": string,
    "suggested_correction": string (if is_valid is false)
}

Example Response:
{
    "is_valid": true,
    "confidence": 0.95,
    "explanation": "Correctly mapped 'magic' to 'Double Ristretto Flat White'",
    "suggested_correction": null
}
"""

In [212]:
import google.generativeai as genai
import os
MODEL = "gemini-2.0-flash-exp"
api_key = os.getenv('GEMINI_API_KEY')
if not api_key:
    raise ValueError("GEMINI_API_KEY environment variable not found")

genai.configure(api_key = api_key)
model = genai.GenerativeModel(MODEL)
judgeModel = genai.GenerativeModel(MODEL)


In [213]:
# get user order
USER_ORDER = "large mocha"
PROMPT = generatePrompt(data,USER_ORDER)
response = model.generate_content(PROMPT)
print(response.text)
judgeResponse = judgeModel.generate_content(generateJudgePrompt(response,data,USER_ORDER))
print(judgeResponse)

Input Order: large mocha
Output Format: [5] large Latte

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "```json\n{\n    \"is_valid\": false,\n    \"confidence\": 0.2,\n    \"explanation\": \"The customer ordered a 'large mocha'. A mocha is a chocolate-flavored latte, and not simply a latte. The size is correctly captured as 'large'.\",\n     \"suggested_correction\": \"[5] large Latte with chocolate\"\n}\n```\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
            },
    

In [214]:
# parse the output to only get the product
import re

def extract_number(text):
    # Find first number in string
    match = re.search(r'\d+', text)
    if match:
        return match.group()
    return None

# Example usage
text = response.text
output_id = extract_number(text)
print(output_id)

5
